In [1]:
# Source: https://data.world/jonloyens/inside-airbnb-austin
# dataset created on 2016-07-15

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
# import nltk
# nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/richardkim/nltk_data...


True

In [112]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from nltk.sentiment import SentimentIntensityAnalyzer

In [148]:
#importing all the data

# calendar = pd.read_csv('C:\\Users\\Richard\\Desktop\\data_airbnb\\calendar.csv')
# listings = pd.read_csv('C:\\Users\\Richard\\Desktop\\data_airbnb\\listings.csv')
# listings_summary = pd.read_csv('C:\\Users\\Richard\\Desktop\\data_airbnb\\listings_summary.csv')
# reviews = pd.read_csv('C:\\Users\\Richard\\Desktop\\data_airbnb\\reviews.csv')
# # neighbourhoods = pd.read_csv('C:\\Users\\Richard\\Desktop\\data_airbnb\\neighbourhoods.csv')
# # neighbourhoods_2 = pd.read_csv('C:\\Users\\Richard\\Desktop\\data_airbnb\\neighbourhoods_2.csv')
# # reviews_summary = pd.read_csv('C:\\Users\\Richard\\Desktop\\data_airbnb\\reviews_summary.csv')

calendar = pd.read_csv('./data/calendar.csv')
listings = pd.read_csv('./data/listings.csv')
listings_summary = pd.read_csv('./data/listings_summary.csv')
reviews = pd.read_csv('./data/reviews.csv')

In [149]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,location
0,72635,https://www.airbnb.com/rooms/72635,20151107173015,2015-11-08,"3 Private Bedrooms, SW Austin",Conveniently located 10-15 from downtown in SW...,"We have three spare bedrooms, each with a quee...",Conveniently located 10-15 from downtown in SW...,none,Location and convenience are key. Easy access...,NaN,Unfortunately there is no convenient public tr...,https://a2.muscache.com/ac/pictures/69586491/a...,https://a2.muscache.com/im/pictures/69586491/a...,https://a2.muscache.com/ac/pictures/69586491/a...,https://a2.muscache.com/ac/pictures/69586491/a...,377044,https://www.airbnb.com/users/show/377044,Jason & Marcie,2011-02-04,"Austin, Texas, United States",Private music loving family in SW Austin,NaN,NaN,NaN,False,https://a2.muscache.com/ac/users/377044/profil...,https://a2.muscache.com/ac/users/377044/profil...,Circle C,1.0,1.0,"['email', 'facebook', 'linkedin', 'reviews']",True,False,"West Slaughter Lane, Austin, TX 78749, United ...",Circle C,78739,NaN,Austin,TX,78749.0,Austin,"Austin, TX",US,United States,30.202824,-97.884309,True,House,Private room,6,2.5,1.0,3.0,Real Bed,"{""Cable TV"",Internet,""Wireless Internet"",""Air ...",3600.0,300.0,NaN,NaN,NaN,NaN,1,0.0,2,10,8 months ago,True,0,0,0,240,2015-11-07,1,2011-03-21,2011-03-21,100.0,10.0,10.0,10.0,10.0,10.0,10.0,False,NaN,NaN,False,moderate,False,False,1,0.02,POINT(-97.88430895565078 30.202823795785008)
1,5386323,https://www.airbnb.com/rooms/5386323,20151107173015,2015-11-07,Cricket Trailer,Rent this cool concept trailer that has everyt...,Rental arrangements for this trailer allows yo...,Rent this cool concept trailer that has everyt...,none,We're talking about wherever you'd like in the...,NaN,"Bike, Bus, Metrorail, etc. you name it we've g...",https://a0.muscache.com/ac/pictures/67298119/7...,https://a0.muscache.com/im/pictures/67298119/7...,https://a0.muscache.com/ac/pictures/67298119/7...,https://a0.muscache.com/ac/pictures/67298119/7...,27922117,https://www.airbnb.com/users/show/27922117,Michael,2015-02-17,"Austin, Texas, United States",Family man that loves the great outdoors.,NaN,NaN,NaN,False,https://a1.muscache.com/ac/users/27922117/prof...,https://a1.muscache.com/ac/users/27922117/prof...,Circle C,1.0,1.0,"['email', 'phone', 'google', 'kba']",True,True,"Seneca Falls Loop, Austin, TX 78739, United St...",Circle C,78739,NaN,Austin,TX,78739.0,Austin,"Austin, TX",US,United States,30.199414,-97.900684,True,Camper/RV,Private room,2,0.0,1.0,1.0,Futon,"{""Air Conditioning"",Heating,""Family/Kid Friend...",NaN,99.0,600.0,NaN,1000.0,75.0,1,0.0,3,7,8 months ago,True,29,59,89,364,2015-11-07,0,NaN,NaN,

In [150]:
calendar['listing_id'] = calendar.listing_id.astype(str)
calendar['date'] = pd.to_datetime(calendar.date)

In [165]:
# listings['id'] = listings.id.astype(str)
listings['thumbnail_exists'] = listings.thumbnail_url.isna().apply(lambda x: int(not x))
listings['medium_exists'] = listings.medium_url.isna().apply(lambda x: int(not x))
listings['picture_exists'] = listings.picture_url.isna().apply(lambda x: int(not x))
listings['xl_picture_exists'] = listings.xl_picture_url.isna().apply(lambda x: int(not x))
listings['last_scraped'] = pd.to_datetime(listings.last_scraped)
listings['host_name'] = listings.host_name.astype(str)
listings['host_since'] = pd.to_datetime(listings.host_since)
listings['host_is_superhost'] = listings.host_is_superhost.astype(bool).astype(int)
def host_response_time_cleaning(arg):
    return {np.nan: np.nan, 'within an hour': 0, 'within a few hours': 1, 'within a day': 2, 'a few days or more': 3}[arg]
listings['host_response_time_numeric'] = listings.host_response_time.apply(host_response_time_cleaning)
listings['host_response_time_numeric'].fillna(listings.host_response_time_numeric.mean(), inplace = True)
l = list(map(lambda x: eval(x), listings.host_verifications))
host_verifications_master_list = []
for ll in l:
    try:
        for i in ll:
            if i not in host_verifications_master_list:
                host_verifications_master_list.append(i)
    except TypeError:
        None
# print(host_verifications_master_list)
for verification_method in host_verifications_master_list:
    listings['host_verification_'+verification_method] = listings.host_verifications.apply(lambda x: int(verification_method in x))
listings['host_has_profile_pic'] = listings.host_has_profile_pic.astype(bool).astype(int)
listings['host_identity_verified'] = listings.host_identity_verified.astype(bool).astype(int)
listings['calendar_last_scraped'] = pd.to_datetime(listings.calendar_last_scraped)
listings['first_review'] = pd.to_datetime(listings.first_review)
listings['last_review'] = pd.to_datetime(listings.last_review)
def cancellation_policy_cleaning(arg):
    return {'flexible': 0,
           'moderate': 1,
           'strict': 2,
           'super_strict_30': 3,
           'no_refunds': 4}[arg]
listings['cancellation_policy_numeric'] = listings.cancellation_policy.apply(cancellation_policy_cleaning)
listings['is_real_bed'] = listings.bed_type.apply(lambda x: int(x == 'Real Bed'))
listings['host_response_rate'] = listings.host_response_rate.fillna(0)
listings['host_acceptance_rate'] = listings.host_acceptance_rate.fillna(0)
listings['reviews_per_month'] = listings.reviews_per_month.fillna(0)
def property_type_cleaning(arg):
    if arg not in ['House', 'Apartment']:
        return 'other'
    else:
        return arg
listings['property_type_cleaned'] = listings.property_type.apply(property_type_cleaning)
listings['requires_license'] = listings.requires_license.astype(int)
listings['instant_bookable'] = listings.instant_bookable.astype(int)
listings['require_guest_profile_picture'] = listings.require_guest_profile_picture.astype(int)
listings['require_guest_phone_verification'] = listings.require_guest_phone_verification.astype(int)

basedate = pd.Timestamp('2016-07-15')
listings['days_since_hosting'] = listings.host_since.apply(lambda x: (basedate - x).days)
listings['days_since_last_updated'] = listings.last_scraped.apply(lambda x: (basedate - x).days)


In [152]:
#number of days that the listing was 'sold out' in 2015.
days_sold_per_listing = calendar.loc[~calendar.available].groupby('listing_id').count()[['available']].reset_index()

In [172]:
listings_abbr = listings[['id', 'host_id', 'host_response_rate', 
          'host_acceptance_rate', 'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
          'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', 'zipcode', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_nights',
          'maximum_nights','availability_365', 'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
         'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_value',
         'requires_license', 'instant_bookable', 'require_guest_profile_picture',
         'require_guest_phone_verification', 'calculated_host_listings_count', 'reviews_per_month',
         'thumbnail_exists', 'medium_exists', 'picture_exists', 'xl_picture_exists', 'host_response_time_numeric', 
        'host_verification_email', 'host_verification_facebook', 'host_verification_linkedin', 
        'host_verification_reviews', 
          'host_verification_phone', 'host_verification_google', 'host_verification_kba', 
          'host_verification_jumio', 'host_verification_sent_id', 'host_verification_manual_online', 
          'host_verification_manual_offline', 'host_verification_photographer', 'host_verification_amex', 
          'cancellation_policy_numeric', 'is_real_bed', 'days_since_hosting', 'days_since_last_updated']]

In [173]:
listings_abbr.head()

,id,host_id,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,zipcode,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_value,requires_license,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,thumbnail_exists,medium_exists,picture_exists,xl_picture_exists,host_response_time_numeric,host_verification_email,host_verification_facebook,host_verification_linkedin,host_verification_reviews,host_verification_phone,host_verification_google,host_verification_kba,host_verification_jumio,host_verification_sent_id,host_verification_manual_online,host_verification_manual_offline,host_verification_photographer,host_verification_amex,cancellation_policy_numeric,is_real_bed,days_since_hosting,days_since_last_updated
0,72635,377044,0.00,0.00,0,1.0,1.0,1,0,78739,78749.0,6,2.5,1.0,3.0,300.0,2,10,240,1,100.0,10.0,10.0,10.0,10.0,10.0,0,0,0,0,1,0.02,1,1,1,1,0.756045,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1988.0,250
1,5386323,27922117,0.00,0.00,0,1.0,1.0,1,1,78739,78739.0,2,0.0,1.0,1.0,99.0,3,7,364,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0.00,1,1,1,1,0.756045,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,514.0,251
2,8826517,46230038,0.00,0.00,0,2.0,2.0,1,1,78739,78739.0,2,1.5,1.0,1.0,100.0,1,1125,365,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,2,0.00,1,1,1,1,0.756045,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,279.0,251
3,8828616,46230038,0.00,0.00,0,2.0,2.0,1,1,78739,78739.0,2,1.5,1.0,1.0,100.0,1,1125,365,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,2,0.00,1,1,1,1,0.756045,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,279.0,250
4,8536913,31148752,0.98,0.81,0,301.0,301.0,1,1,78739,78739.0,6,2.0,3.0,3.0,599.0,2,1125,358,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,2,0.00,1,1,1,1,1.000000,1,0,1,1,1,0,0,1,0,0,0,0,0,2,1,459.0,250


In [174]:
#doing One-hot Encoding for all categorical values

listings_property_type_OHE = pd.get_dummies(listings.property_type_cleaned)
listings_room_type_OHE = pd.get_dummies(listings.room_type)
listings2 = pd.concat([listings_abbr, listings_property_type_OHE, listings_room_type_OHE], axis = 1).head()

In [175]:
listings2.dtypes

id                                    int64
host_id                               int64
host_response_rate                  float64
host_acceptance_rate                float64
host_is_superhost                     int64
host_listings_count                 float64
host_total_listings_count           float64
host_has_profile_pic                  int64
host_identity_verified                int64
neighbourhood_cleansed                int64
zipcode                             float64
accommodates                          int64
bathrooms                           float64
bedrooms                            float64
beds                                float64
price                               float64
minimum_nights                        int64
maximum_nights                        int64
availability_365                      int64
number_of_reviews                     int64
review_scores_rating                float64
review_scores_accuracy              float64
review_scores_cleanliness       

In [176]:
len(listings2.columns)

60

In [178]:
sia = SentimentIntensityAnalyzer()

reviews['date'] = pd.to_datetime(reviews.date)
def comments_sentiment_neg_cleaning(arg):
    try:
        return sia.polarity_scores(arg)['neg']
    except:
        return -1
def comments_sentiment_neu_cleaning(arg):
    try:
        return sia.polarity_scores(arg)['neu']
    except:
        return -1
def comments_sentiment_pos_cleaning(arg):
    try:
        return sia.polarity_scores(arg)['pos']
    except:
        return -1
reviews['comments_sentiment_neg'] = reviews.comments.apply(comments_sentiment_neg_cleaning)
reviews['comments_sentiment_neu'] = reviews.comments.apply(comments_sentiment_neu_cleaning)
reviews['comments_sentiment_pos'] = reviews.comments.apply(comments_sentiment_pos_cleaning)

In [190]:
#dropping 11 out of 62371 rows where Sentiment Intensity Analyzer Fails
reviews = reviews.loc[(reviews.comments_sentiment_neg != -1) & \
           (reviews.comments_sentiment_neu != -1) & \
           (reviews.comments_sentiment_pos != -1)]

In [191]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,comments_sentiment_neg,comments_sentiment_neu,comments_sentiment_pos
0,2390109,22208895,2014-11-01,4358294,Inessa,I had a great time staying here! The room is h...,0.000,0.792,0.208
1,2390109,22619785,2014-11-10,12361335,Colibri,What a wonderful space in a beautiful & quiet ...,0.000,0.730,0.270
2,2390109,23234873,2014-11-25,6359216,Giuseppe,The reservation was canceled 107 days before a...,0.000,1.000,0.000
3,2390109,23338508,2014-11-29,23638127,Sophia,When I saw the posting and how clean and simpl...,0.017,0.815,0.168
4,2390109,24989361,2015-01-04,3088100,Taylor,This is one of the first times we've been hesi...,0.021,0.794,0.186


In [192]:
reviews[['listing_id', 'comments_sentiment_neg', 'comments_sentiment_neu', 'comments_sentiment_pos']]\
.groupby('listing_id').mean()

,comments_sentiment_neg,comments_sentiment_neu,comments_sentiment_pos
listing_id,,,
1078,0.008343,0.690147,0.301510
2265,0.026250,0.714333,0.259333
5245,0.002500,0.648000,0.349500
5456,0.011952,0.679112,0.308917
5769,0.010700,0.642911,0.346383
6251,0.010980,0.657842,0.331188
6413,0.005000,0.756211,0.238789
6448,0.008447,0.681235,0.310294
7679,0.006481,0.671013,0.322532
